In [14]:
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as disp
import folium

In [2]:
ee.Initialize()

In [3]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWh2CytQUgAINvRzuWthwI23mqlFeMwIdMi7gFOUYH_HQCtRHZ9wVpE

Successfully saved authorization token.


In [9]:
africa_geoJSON =  {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              17.832336, 
              -19.960908
            ],
            [
              17.832336,
              56.627625
            ],
            [
              -34.554214,
              56.627625
            ],
            [
              -34.554214,
              -19.960908
            ]
          ]
        ]
      }
    }
  ]
}

In [10]:
coords = africa_geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [101]:
dfo_floods = ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1")
dfo_floods = dfo_floods.select(['flooded', 'jrc_perm_water'])

In [117]:
def createTimeBand(img):
    year = ee.Date(img.get('system:time_start')).get('year').subtract(1991)
    mask = img.select('flooded').gt(0).And(img.select('jrc_perm_water').eq(0))
    return ee.Image(year).byte().addBands(img.updateMask(mask)).select(['constant', 'flooded'])

In [118]:
dfo_flooded = dfo_floods.map(createTimeBand)

In [120]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data © <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)
 

folium.Map.add_ee_layer = add_ee_layer

In [143]:
flood_map = folium.Map(location = [17.832336,56.627625], zoom_start = 4.5, tiles = 'OpenStreetMap')
flood = dfo_flooded.reduce(ee.Reducer.linearFit())
flood_map.add_ee_layer(flood, params, 'test')
display(flood_map)

In [147]:
dfo_floods_recent = dfo_floods.filterDate('2015-01-01', '2019-01-01')
dfo_flooded_recent = dfo_floods_recent.map(createTimeBand)
recent_flood_map = folium.Map(location = [17.832336,56.627625], zoom_start = 4.5, tiles = 'OpenStreetMap')
recent_flood = dfo_flooded_recent.reduce(ee.Reducer.linearFit())
recent_flood_map.add_ee_layer(recent_flood, params, 'test')
display(recent_flood_map)